# ETL Project

Data information was taken from page: https://catalog.data.gov/dataset?res_format=JSON&tags=wine
 
Wine Licenses in NY State were reviewed, and uploaded into an SQL database.

Two datasets with Active Licenses and Application for License were taken to review: per class of licence, how many are received, issued and expired, per license type, zip code and city.


---

### Dependencies

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [2]:
# Dependencies for DB Connection
from sqlalchemy import create_engine
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float
# To push the objects made and query the server we use a Session object
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

### Explore Active_Licenses csv file

In [3]:
# First, get all information from the Active Licenses csv file
ny_active_licenses = os.path.join("Liquor_Authority_Quarterly_List_of_Active_Licenses.csv")
#Wds = ("./Liquor_Authority_Quarterly_List_of_Active_Licenses.csv")
ny_active_licenses_df = pd.read_csv(ny_active_licenses)
ny_active_licenses_df.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,License Serial Number,License Type Name,License Class Code,License Type Code,Agency Zone Office Name,Agency Zone Office Number,County Name (Licensee),Premises Name,Doing Business As (DBA),Actual Address of Premises (Address1),...,City,State,Zip,License Certificate Number,License Original Issue Date,License Effective Date,License Expiration Date,Latitude,Longitude,Location
0,1000080,GROCERY STORE BEER,122.0,A,New York City,1,BRONX,ERIDANIA CORP,1888 WALTON GROCERY,1888 WALTON AVE,...,BRONX,NY,10452,913175,02/07/2019,02/06/2019,02/28/2022,40.849430,-73.909261,"(40.8494297, -73.90926083)"
1,1000090,GROCERY STORE BEER,122.0,A,New York City,1,BRONX,1098 MINI MARKET INC,JACOBO GROCERY & DELI,1098 ANDERSON AVENUE,...,BRONX,NY,10452,888708,08/31/2004,03/01/2017,02/29/2020,40.834710,-73.925238,"(40.83471022, -73.92523844)"
2,1000099,GROCERY STORE BEER,122.0,A,New York City,1,BRONX,J & A FRUIT & GROCERY CORP,J & A FRUIT & GROCERY CTR,11 17 E 171ST STREET,...,BRONX,NY,10452,893397,09/20/2007,11/01/2017,10/31/2020,40.841277,-73.916582,"(40.84127748, -73.91658233)"
3,1000167,GROCERY STORE BEER,122.0,A,New York City,1,BRONX,ALTAGRACIA GROCERY CORP,NaN,10 12 E 176TH STREET,...,BRONX,NY,10453,892433,NaN,07/01/2017,06/30/2020,NaN,NaN,NaN
4,1000207,GROCERY STORE BEER,122.0,A,New York City,1,BRONX,48 BURNSIDE FOOD CORP,FOOD DYNASTY,40 48 WEST BURNSIDE AVE,...,BRONX,NY,10453,908509,10/15/1997,10/01/2018,09/30/2021,40.853900,-73.909440,"(40.85390041, -73.9094398)"


In [4]:
# Check data types of the dataframe
ny_active_licenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51235 entries, 0 to 51234
Data columns (total 21 columns):
License Serial Number                        51235 non-null int64
License Type Name                            51235 non-null object
License Class Code                           51189 non-null float64
License Type Code                            51235 non-null object
Agency Zone Office Name                      51235 non-null object
Agency Zone Office Number                    51235 non-null int64
County Name (Licensee)                       51235 non-null object
Premises Name                                51235 non-null object
Doing Business As (DBA)                      35795 non-null object
Actual Address of Premises (Address1)        51235 non-null object
Additional Address Information (Address2)    11154 non-null object
City                                         51235 non-null object
State                                        51235 non-null object
Zip                  

In [5]:
# Select only the desired columns to work with
ny_active_licenses_df = ny_active_licenses_df[['License Serial Number','License Type Name','License Class Code','License Type Code','City','Zip','License Certificate Number','License Original Issue Date','License Effective Date','License Expiration Date']]
ny_active_licenses_df.head()

,License Serial Number,License Type Name,License Class Code,License Type Code,City,Zip,License Certificate Number,License Original Issue Date,License Effective Date,License Expiration Date
0,1000080,GROCERY STORE BEER,122.0,A,BRONX,10452,913175,02/07/2019,02/06/2019,02/28/2022
1,1000090,GROCERY STORE BEER,122.0,A,BRONX,10452,888708,08/31/2004,03/01/2017,02/29/2020
2,1000099,GROCERY STORE BEER,122.0,A,BRONX,10452,893397,09/20/2007,11/01/2017,10/31/2020
3,1000167,GROCERY STORE BEER,122.0,A,BRONX,10453,892433,NaN,07/01/2017,06/30/2020
4,1000207,GROCERY STORE BEER,122.0,A,BRONX,10453,908509,10/15/1997,10/01/2018,09/30/2021


In [6]:
# Check for non null data
ny_active_licenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51235 entries, 0 to 51234
Data columns (total 10 columns):
License Serial Number          51235 non-null int64
License Type Name              51235 non-null object
License Class Code             51189 non-null float64
License Type Code              51235 non-null object
City                           51235 non-null object
Zip                            51235 non-null object
License Certificate Number     51235 non-null int64
License Original Issue Date    46633 non-null object
License Effective Date         51196 non-null object
License Expiration Date        51235 non-null object
dtypes: float64(1), int64(2), object(7)
memory usage: 3.9+ MB


In [7]:
# Review data types of the dataframe
ny_active_licenses_df.dtypes

License Serial Number            int64
License Type Name               object
License Class Code             float64
License Type Code               object
City                            object
Zip                             object
License Certificate Number       int64
License Original Issue Date     object
License Effective Date          object
License Expiration Date         object
dtype: object

### Explore New_Applications csv file

In [9]:
applications = os.path.join("NYS_Liquor_Authority_New_Applications_Received.csv")
applications_df = pd.read_csv(applications)
applications_df.head()

,License Serial Number,License Type Name,License Class Code,License Type Code,Agency Zone Office Number,Agency Zone Office Name,County Name (Licensee),Premises Name,Doing Business As (DBA),Actual Address of Premises (Address1),Additional Address Information (Address2),City,State,ZIP,License Certificate Number,License Received Date
0,1302914,ON-PREMISES LIQUOR,252.0,OP,1,New York City,NEW YORK,53 LAPIDAR INC,NaN,53 STONE ST,NaN,NEW YORK,NY,10004,NaN,03/05/2019
1,1304304,ON-PREMISES LIQUOR,252.0,OP,1,New York City,QUEENS,WOLFE LIC INC,2 DIVE 4,33-10 36TH AVE,NaN,LONG ISLAND CITY,NY,11106,888417.0,01/17/2019
2,1304478,RESTAURANT WINE,341.0,RW,1,New York City,NEW YORK,FLAT TOP INC,FRANKIE & MATT,508 COLUMBUS AVE,NaN,NEW YORK,NY,10024,NaN,03/22/2019
3,1306781,ON-PREMISES LIQUOR,252.0,OP,1,New York City,QUEENS,EL ENCANTO CENTRO AMERICANO RESTAURANT INC,EL ENCANTO CENTRO AMERICANO RESTAURANT,14912 JAMAICA AVE,NaN,JAMAICA,NY,11435,888668.0,03/19/2019
4,1308127,"GROCERY BEER, WINE PROD",122.0,AX,1,New York City,SUFFOLK,ELITE GROUP HOLBROOK INC,NaN,4815 VETERANS MEMORIAL HWY,NaN,HOLBROOK,NY,11741,NaN,03/13/2019


In [10]:
applications_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719 entries, 0 to 1718
Data columns (total 16 columns):
License Serial Number                        1719 non-null int64
License Type Name                            1719 non-null object
License Class Code                           1718 non-null float64
License Type Code                            1719 non-null object
Agency Zone Office Number                    1719 non-null int64
Agency Zone Office Name                      1719 non-null object
County Name (Licensee)                       1719 non-null object
Premises Name                                1719 non-null object
Doing Business As (DBA)                      999 non-null object
Actual Address of Premises (Address1)        1719 non-null object
Additional Address Information (Address2)    222 non-null object
City                                         1719 non-null object
State                                        1719 non-null object
ZIP                                   

In [11]:
# Select only the desired columns to work with
applications_df = applications_df[['License Serial Number','License Type Name','License Class Code','License Type Code','City','ZIP','License Certificate Number','License Received Date']]
applications_df.head()

,License Serial Number,License Type Name,License Class Code,License Type Code,City,ZIP,License Certificate Number,License Received Date
0,1302914,ON-PREMISES LIQUOR,252.0,OP,NEW YORK,10004,NaN,03/05/2019
1,1304304,ON-PREMISES LIQUOR,252.0,OP,LONG ISLAND CITY,11106,888417.0,01/17/2019
2,1304478,RESTAURANT WINE,341.0,RW,NEW YORK,10024,NaN,03/22/2019
3,1306781,ON-PREMISES LIQUOR,252.0,OP,JAMAICA,11435,888668.0,03/19/2019
4,1308127,"GROCERY BEER, WINE PROD",122.0,AX,HOLBROOK,11741,NaN,03/13/2019


In [12]:
# Check for non null data
applications_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719 entries, 0 to 1718
Data columns (total 8 columns):
License Serial Number         1719 non-null int64
License Type Name             1719 non-null object
License Class Code            1718 non-null float64
License Type Code             1719 non-null object
City                          1719 non-null object
ZIP                           1719 non-null int64
License Certificate Number    160 non-null float64
License Received Date         1719 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 107.5+ KB


In [13]:
# Check for datatypes of the dataframe
applications_df.dtypes

License Serial Number           int64
License Type Name              object
License Class Code            float64
License Type Code              object
City                           object
ZIP                             int64
License Certificate Number    float64
License Received Date          object
dtype: object

### Renaming Columns in both DataFrames

In [14]:
ny_active_licenses_sql = ny_active_licenses_df.rename(columns={
    'License Serial Number':'license_serial_number',
    'License Type Name':'license_type_name',
    'License Class Code':'license_class_code',
    'License Type Code':'license_type_code',
    'City':'city',
    'Zip':'zip',
    'License Certificate Number':'license_certificate_number',
    'License Original Issue Date':'license_original_issue_date',
    'License Effective Date':'license_effective_date',
    'License Expiration Date':'license_expiration_date'
}).copy()
ny_active_licenses_sql.head()

,license_serial_number,license_type_name,license_class_code,license_type_code,city,zip,license_certificate_number,license_original_issue_date,license_effective_date,license_expiration_date
0,1000080,GROCERY STORE BEER,122.0,A,BRONX,10452,913175,02/07/2019,02/06/2019,02/28/2022
1,1000090,GROCERY STORE BEER,122.0,A,BRONX,10452,888708,08/31/2004,03/01/2017,02/29/2020
2,1000099,GROCERY STORE BEER,122.0,A,BRONX,10452,893397,09/20/2007,11/01/2017,10/31/2020
3,1000167,GROCERY STORE BEER,122.0,A,BRONX,10453,892433,NaN,07/01/2017,06/30/2020
4,1000207,GROCERY STORE BEER,122.0,A,BRONX,10453,908509,10/15/1997,10/01/2018,09/30/2021


In [15]:
applications_sql = applications_df.rename(columns={
    'License Serial Number':'license_serial_number',
    'License Type Name':'license_type_name',
    'License Class Code':'license_class_code',
    'License Type Code':'license_type_code',
    'City':'city',
    'ZIP':'zip',
    'License Certificate Number':'license_certificate_number',
    'License Received Date':'license_received_date'
}).copy()
applications_sql.head()

,license_serial_number,license_type_name,license_class_code,license_type_code,city,zip,license_certificate_number,license_received_date
0,1302914,ON-PREMISES LIQUOR,252.0,OP,NEW YORK,10004,NaN,03/05/2019
1,1304304,ON-PREMISES LIQUOR,252.0,OP,LONG ISLAND CITY,11106,888417.0,01/17/2019
2,1304478,RESTAURANT WINE,341.0,RW,NEW YORK,10024,NaN,03/22/2019
3,1306781,ON-PREMISES LIQUOR,252.0,OP,JAMAICA,11435,888668.0,03/19/2019
4,1308127,"GROCERY BEER, WINE PROD",122.0,AX,HOLBROOK,11741,NaN,03/13/2019


### Filling NaN values with a valid one

In [16]:
# Filling values in the active licenses dataframe
ny_active_licenses_sql['license_original_issue_date'].fillna(value='01/01/1500', inplace=True)
ny_active_licenses_sql['license_effective_date'].fillna(value='01/01/1500', inplace=True)
ny_active_licenses_sql['license_class_code'].fillna(value=0.01, inplace=True)

In [17]:
ny_active_licenses_sql.head()

,license_serial_number,license_type_name,license_class_code,license_type_code,city,zip,license_certificate_number,license_original_issue_date,license_effective_date,license_expiration_date
0,1000080,GROCERY STORE BEER,122.0,A,BRONX,10452,913175,02/07/2019,02/06/2019,02/28/2022
1,1000090,GROCERY STORE BEER,122.0,A,BRONX,10452,888708,08/31/2004,03/01/2017,02/29/2020
2,1000099,GROCERY STORE BEER,122.0,A,BRONX,10452,893397,09/20/2007,11/01/2017,10/31/2020
3,1000167,GROCERY STORE BEER,122.0,A,BRONX,10453,892433,01/01/1500,07/01/2017,06/30/2020
4,1000207,GROCERY STORE BEER,122.0,A,BRONX,10453,908509,10/15/1997,10/01/2018,09/30/2021


In [18]:
# Check that NaN values are dismissed.
ny_active_licenses_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51235 entries, 0 to 51234
Data columns (total 10 columns):
license_serial_number          51235 non-null int64
license_type_name              51235 non-null object
license_class_code             51235 non-null float64
license_type_code              51235 non-null object
city                           51235 non-null object
zip                            51235 non-null object
license_certificate_number     51235 non-null int64
license_original_issue_date    51235 non-null object
license_effective_date         51235 non-null object
license_expiration_date        51235 non-null object
dtypes: float64(1), int64(2), object(7)
memory usage: 3.9+ MB


In [19]:
# Filling values in the requested applications licenses dataframe
applications_sql['license_class_code'].fillna(value=1, inplace=True)
applications_sql['license_certificate_number'].fillna(value=2, inplace=True)

In [20]:
# Check that NaN values are dismissed.
applications_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719 entries, 0 to 1718
Data columns (total 8 columns):
license_serial_number         1719 non-null int64
license_type_name             1719 non-null object
license_class_code            1719 non-null float64
license_type_code             1719 non-null object
city                          1719 non-null object
zip                           1719 non-null int64
license_certificate_number    1719 non-null float64
license_received_date         1719 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 107.5+ KB


### Pass DataFrames to SQL

First we pass individual dataframes to SQL individual tables.

In [21]:
# FIRST CREATE SQL SCHEMA AND DB IN WORKBENCH
# Create connection engine
connection_string = "root:Mysqlmypassword@localhost/ny_licences_wines_db"
engine = create_engine(f'mysql://{connection_string}')

# Confirm tables
engine.table_names()

['active_licenses', 'all_licenses', 'license_class_type', 'requested_licenses']

In [22]:
# Pass first DF to SQL
ny_active_licenses_sql.to_sql(name='active_licenses', con=engine, if_exists='append', index=False)

In [23]:
# Pass second DF to SQL
applications_sql.to_sql(name='requested_licenses', con=engine, if_exists='append', index=False)

In [24]:
applications_sql.count()

license_serial_number         1719
license_type_name             1719
license_class_code            1719
license_type_code             1719
city                          1719
zip                           1719
license_certificate_number    1719
license_received_date         1719
dtype: int64

### Merging both DataFrames

In [27]:
# Merge both DataFrames
merged_dfs = ny_active_licenses_sql.merge(applications_sql, how='outer')
# OR: merged_dfs = pd.merge(ny_active_licenses_sql, applications_sql, how='outer')
merged_dfs.head()

,license_serial_number,license_type_name,license_class_code,license_type_code,city,zip,license_certificate_number,license_original_issue_date,license_effective_date,license_expiration_date,license_received_date
0,1000080,GROCERY STORE BEER,122.0,A,BRONX,10452,913175.0,02/07/2019,02/06/2019,02/28/2022,NaN
1,1000090,GROCERY STORE BEER,122.0,A,BRONX,10452,888708.0,08/31/2004,03/01/2017,02/29/2020,NaN
2,1000099,GROCERY STORE BEER,122.0,A,BRONX,10452,893397.0,09/20/2007,11/01/2017,10/31/2020,NaN
3,1000167,GROCERY STORE BEER,122.0,A,BRONX,10453,892433.0,01/01/1500,07/01/2017,06/30/2020,NaN
4,1000207,GROCERY STORE BEER,122.0,A,BRONX,10453,908509.0,10/15/1997,10/01/2018,09/30/2021,NaN


In [28]:
# Check for NaN values after merging
merged_dfs.count()

license_serial_number          52915
license_type_name              52915
license_class_code             52915
license_type_code              52915
city                           52915
zip                            52915
license_certificate_number     52915
license_original_issue_date    51235
license_effective_date         51235
license_expiration_date        51235
license_received_date           1719
dtype: int64

In [29]:
# Replace NaN values.
merged_dfs['license_original_issue_date'].fillna(value='01/01/1500', inplace=True)
merged_dfs['license_effective_date'].fillna(value='01/01/1500', inplace=True)
merged_dfs['license_expiration_date'].fillna(value='01/01/1500', inplace=True)
merged_dfs['license_received_date'].fillna(value='01/01/1500', inplace=True)

## LICENSES CLASS CODES

Now let's create a new dataframe and table containing only the information for the licenses types that are observed in the datasets.

To do this we extract only the columns for the class code and the type of license belonging to that code. Then we pass that information to a new table and store that information into the SQL database.

In [30]:
class_type_df = merged_dfs[['license_class_code', 'license_type_name']]
class_type_df.head()

,license_class_code,license_type_name
0,122.0,GROCERY STORE BEER
1,122.0,GROCERY STORE BEER
2,122.0,GROCERY STORE BEER
3,122.0,GROCERY STORE BEER
4,122.0,GROCERY STORE BEER


In [31]:
class_codes_df = class_type_df.drop_duplicates()
class_codes_df.head()

,license_class_code,license_type_name
0,122.0,GROCERY STORE BEER
9,122.0,"GROCERY BEER, WINE PROD"
95,141.0,EATING PLACE BEER
115,104.0,WHOLESALE CIDER
116,103.0,WHOLESALE BEER(C)


In [32]:
class_codes_df.count()

license_class_code    94
license_type_name     94
dtype: int64

In [33]:
# Drop duplicates on LICENSE CLASS CODE
class_codes_df2 = class_codes_df.iloc[1:]
class_codes_df2.head()

,license_class_code,license_type_name
9,122.0,"GROCERY BEER, WINE PROD"
95,141.0,EATING PLACE BEER
115,104.0,WHOLESALE CIDER
116,103.0,WHOLESALE BEER(C)
122,243.0,CLUB LIQUOR


In [34]:
# Drop duplicates on LICENSE CLASS CODE
class_codes_df = class_codes_df.drop_duplicates(subset=['license_class_code'])

In [35]:
class_codes_df['license_class_code'].value_counts()
class_codes_df.count()

license_class_code    82
license_type_name     82
dtype: int64

In [36]:
# Pass to SQL table the information for license class codes and license names,
# with license class code as primary key.
class_codes_df.to_sql(name='license_class_type', con=engine, if_exists='append', index=False)

In [37]:
# Verify that there is only one code per license code
class_codes_df['license_class_code'].value_counts()

0.01      1
144.00    1
541.00    1
543.00    1
540.00    1
344.00    1
240.00    1
143.00    1
301.00    1
302.00    1
244.00    1
548.00    1
544.00    1
305.00    1
443.00    1
260.00    1
322.00    1
345.00    1
256.00    1
251.00    1
303.00    1
313.00    1
203.00    1
141.00    1
104.00    1
103.00    1
243.00    1
105.00    1
222.00    1
246.00    1
         ..
346.00    1
140.00    1
107.00    1
202.00    1
401.00    1
255.00    1
545.00    1
340.00    1
1.00      1
515.00    1
352.00    1
208.00    1
257.00    1
342.00    1
253.00    1
206.00    1
207.00    1
210.00    1
347.00    1
205.00    1
348.00    1
110.00    1
549.00    1
106.00    1
309.00    1
130.00    1
307.00    1
542.00    1
120.00    1
122.00    1
Name: license_class_code, Length: 82, dtype: int64